## Train

## Importing Libraries

In [1]:
import pkg_resources

REQUIRED_PACKAGES = [
    'tensorflow', 'keras-facenet', 'sklearn', 'scikit-learn'
]

for package in REQUIRED_PACKAGES:
    try:
        dist = pkg_resources.get_distribution(package)
        print('{} ({}) is already installed'.format(dist.key, dist.version))
    except pkg_resources.DistributionNotFound:
        !pip install {package}

tensorflow (2.12.0) is already installed
keras-facenet (0.3.2) is already installed
sklearn (0.0.post5) is already installed
scikit-learn (1.2.2) is already installed


In [2]:
# Only for jupyter-notebook/Colab
!pip uninstall -y opencv-python
!pip install opencv-python-headless


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


## Extract Faces

In [3]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
import cv2
from mtcnn.mtcnn import MTCNN

2023-06-06 10:53:38.807915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# extract a single face from a given photograph
def extract_face(filename, detector, required_size=(224, 224)):
    # load image from file
    image = cv2.imread(filename)
    # convert to RGB, if needed
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # resize pixels to the model size
    if min(image.shape[:2]) < 63:
        image = cv2.resize(image, (320, 320))

    # Set input size for Face detection model
    detector.setInputSize((image.shape[1], image.shape[0]))

    # Detect faces in the image using YuNet
    _, results = detector.detect(image)
    
    face_array = None
    if results is not None:
        # extract the bounding box from the first face
        x1, y1, width, height = tuple(map(int, results[0][:4]))
        
        # bug fix
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = image[y1:y2, x1:x2]
        # resize pixels to the model size
        face_array = cv2.resize(face, required_size)
    
    return face_array

In [5]:
# load images and extract faces for all images in a directory
def load_faces(directory, detector):
    faces = list()
    # enumerate files
    for filename in listdir(directory):
        # path
        path = directory + filename
        # get face
        face = extract_face(path, detector)
        if face is not None:
            # store
            faces.append(face)
    return faces

In [6]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory, detector):
    X, y = list(), list()
    # enumerate folders, on per class
    for subdir in listdir(directory):
        # path
        path = directory + subdir + '/'
        # skip any files that might be in the dir
        if not isdir(path):
            continue
        # load all faces in the subdirectory
        faces = load_faces(path, detector)
        # create labels
        labels = [subdir for _ in range(len(faces))]
        # summarize progress
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        # store
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [7]:
# Load the YuNet face detection model
detector = cv2.FaceDetectorYN.create("detector/face_detection_yunet_2022mar.onnx", "", (320, 320))

# load train dataset
trainX, trainy = load_dataset('images/train/', detector)
print(trainX.shape, trainy.shape)

# load test dataset
testX, testy = load_dataset('images/val/', detector)

# save arrays to one file in compressed format
savez_compressed('ss.npz', trainX, trainy, testX, testy)

>loaded 3 examples for class: 1685_Anjana_Om_Kashyap
>loaded 3 examples for class: 1686_Barkha_Dutt
>loaded 3 examples for class: 1687_Ravish_Kumar
>loaded 3 examples for class: 1688_Arnab_Goswami
>loaded 3 examples for class: person1
>loaded 1 examples for class: person2
>loaded 1 examples for class: person3
(17, 224, 224, 3) (17,)
>loaded 1 examples for class: 1685_Anjana_Om_Kashyap
>loaded 2 examples for class: 1686_Barkha_Dutt
>loaded 2 examples for class: 1687_Ravish_Kumar
>loaded 2 examples for class: 1688_Arnab_Goswami


## Augmentation

In [8]:
!pip install albumentations


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [9]:
import albumentations as A
from numpy import load
import random

In [10]:
# load the face dataset
data = load('ss.npz')

trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

In [11]:
transform = A.Compose([
    # A.RandomCrop(width=200, height=200),
    # A.HorizontalFlip(p=0.5),
    # A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.2),
    A.GaussNoise(var_limit=(10.0, 50.0), mean=0, p=0.5)
])

In [12]:
total_samples_after_aug = 2*len(trainX)
choices = random.choices(range(len(trainX)), k=total_samples_after_aug)

trainX_aug = []
trainy_aug = []

for x in choices:
    trainX_aug.append(transform(image=trainX[x])["image"])
    trainy_aug.append(trainy[x])

In [13]:
# save arrays to one file in compressed format
savez_compressed('ss-aug.npz', trainX_aug, trainy_aug, testX, testy)

## Embeddings

In [14]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
from keras_facenet import FaceNet
from tensorflow import keras
from sklearn.svm import SVC
from tensorflow import keras
import tensorflow as tf

In [15]:
embedder = FaceNet()

2023-06-06 10:53:47.848967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-06 10:53:47.875541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-06 10:53:47.876160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-06 10:53:47.878494: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-06 10:53:47.878917: I tensorflow/compile

In [16]:
# get the face embedding for one face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    detections = model.extract(face_pixels, threshold = 0.95)
    # standardize pixel values across channels (global)
    #mean, std = face_pixels.mean(), face_pixels.std()
    #face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    # yhat = model.predict(samples)
    yhat = model.embeddings(samples)

    return yhat[0]

In [17]:
# load the face dataset
data = load('ss-aug.npz')

trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model
# model = load_model('facenet_keras.h5')
# model.summary()
model = embedder
# model = SVC(kernel='linear', probability=True)
# model = load_model('face-rec_Google.npz')
# model = load('test.npz')
# model = tf.keras.applications.ResNet50(weights='imagenet')

print('Loaded Model')
# convert each face in the train set to an embedding

Loaded:  (34, 224, 224, 3) (34,) (7, 224, 224, 3) (7,)
Loaded Model


In [18]:
newTrainX = list()
for face_pixels in trainX:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
    embedding = get_embedding(model, face_pixels)
    newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('ss-embd.npz', newTrainX, trainy, newTestX, testy)

2023-06-06 10:53:55.127590: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


1/1 [==============================] - 4s 4s/step


2023-06-06 10:53:57.034580: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 0s 28ms/step
(34, 512)
1/1 [==============================] - 0s 31ms/step
(7, 512)


## KNN

In [19]:
# load the embedding dataset
data = load('ss-embd.npz')

trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

In [20]:
# Create and train the KNN classifier
from sklearn import neighbors
import pickle

knn_clf = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance')
knn_clf.fit(trainX, trainy)

KNeighborsClassifier(weights='distance')

In [21]:
# Evaluate classifier with test set
y_pred = knn_clf.predict(testX)

knn_clf.score(testX,testy)

1.0

In [22]:
# Evaluate metrics
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(testy, y_pred))
print(confusion_matrix(testy, y_pred))

                        precision    recall  f1-score   support

1685_Anjana_Om_Kashyap       1.00      1.00      1.00         1
      1686_Barkha_Dutt       1.00      1.00      1.00         2
     1687_Ravish_Kumar       1.00      1.00      1.00         2
    1688_Arnab_Goswami       1.00      1.00      1.00         2

              accuracy                           1.00         7
             macro avg       1.00      1.00      1.00         7
          weighted avg       1.00      1.00      1.00         7

[[1 0 0 0]
 [0 2 0 0]
 [0 0 2 0]
 [0 0 0 2]]


In [23]:
# Save the trained KNN classifier
model_save_path = "trained_face_knn.clf"

with open(model_save_path, 'wb') as f:
    pickle.dump(knn_clf, f)

## Webcam

In [24]:
!pip install openpyxl


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [28]:
import cv2
import numpy as np
from keras.models import load_model
from keras_facenet import FaceNet
from time import time
import pickle
import pandas as pd
import openpyxl
from datetime import datetime

In [29]:
# Load the FaceNet model
facenet_model = FaceNet()

# Load KNN classifier
model_path = ("trained_face_knn.clf")
with open(model_path, 'rb') as f:
    knn_clf = pickle.load(f)

# Load the YuNet face detection model
detector = cv2.FaceDetectorYN.create("detector/face_detection_yunet_2022mar.onnx", "", (320, 320))

In [30]:
# Open the video capture
video_capture = cv2.VideoCapture('test-vids/arnab_goswami_1 - Trim.mp4') # Use 0 for webcam or provide the path to a video file

vid_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
vid_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('outputs/output-yunet.mp4', fourcc, video_capture.get(cv2.CAP_PROP_FPS), 
                      (vid_width, vid_height))

# Set input size for Face detection model
detector.setInputSize((vid_width, vid_height))


# Open the existing logs Excel file
writer = pd.ExcelWriter('logs.xlsx',engine="openpyxl",mode="a",if_sheet_exists="overlay")
writer.workbook = openpyxl.load_workbook('logs.xlsx')


while True:
    # Start timer
    start_time = time()
    
    # Read a frame from the video
    ret, frame = video_capture.read()

    timestamp = datetime.now()

    if not ret:
        break

    # Detect faces in the frame using YuNet
    _, faces = detector.detect(frame)
    
    log_data = []

    # Iterate over detected faces
    if faces is not None:
        for face in faces:
            # Extract the face coordinates
            x, y, w, h = tuple(map(int, face[:4]))

            x, y = max(0, min(x, vid_width)), max(0, min(y, vid_height))
            x_max, y_max = min(x+w, vid_width), min(y+h, vid_height)
    
            # Extract the face ROI from the frame
            face_roi = frame[y:y_max, x:x_max]
    
            # Preprocess the face ROI
            face_roi = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
            face_roi = cv2.resize(face_roi, (224, 224))
            face_roi = face_roi.astype('float32')
            # mean, std = face_roi.mean(), face_roi.std()
            # face_roi = (face_roi - mean) / std
    
            # Convert the preprocessed face ROI to a 4D tensor
            face_tensor = np.expand_dims(face_roi, axis=0)
    
            # Generate embeddings using the FaceNet model
            face_embedding = facenet_model.embeddings(face_tensor)[0]
    
            # Use the KNN model to predict the probability for each class
            proba = knn_clf.predict_proba(np.expand_dims(face_embedding, axis=0))
            print(f"Probability prediction: {proba}")
    
             # Use the KNN model to find distance to the best matches for the test face
            closest_distances = knn_clf.kneighbors(np.expand_dims(face_embedding, axis=0), n_neighbors=1)
            min_distance = closest_distances[0][0][0]
            print(f"Min Distance: {min_distance}")
    
            # Define a threshold for face recognition
            threshold = 0.75
            
            if min_distance < threshold:
                # Identify the recognized face
                recognized_name = knn_clf.classes_[proba.argmax()]
                print(f"Recognized Name: {recognized_name}")
                
                log_data.append((timestamp,recognized_name,max(proba[0])))
    
                # Draw bounding box and label for the recognized face with confidence level
                label = f"{recognized_name} (Prob: {max(proba[0]):.2f})"
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                print(f"Recognized Name: Unknown")
     
                # Draw bounding box and label for unknown face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(frame, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)


    
    # end timer
    time_taken = time() - start_time
    fps = 1/time_taken
    cv2.putText(frame, f'Fps: {round(fps, 3)}', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # print(f'FPS:{fps}')

    df = pd.DataFrame(log_data, columns=['Timestamp', 'Name', 'Probability'])
    # Write the DataFrame to the Excel sheet
    df.to_excel(writer, sheet_name='Sheet1', index=False, header=False, startrow=writer.sheets['Sheet1'].max_row)
    
    # Display the frame
    # cv2.imshow('Video', frame)
    out.write(frame)

    # Exit the loop if 'q' is pressed (commented to work with opencv-python-headless install)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break


# Save the changes to the Excel file
writer.close()
# Release the video capture and close all
video_capture.release()
out.release()

1/1 [==============================] - 2s 2s/step
Probability prediction: [[0.         0.         0.19767275 0.19741588 0.60491137]]
Min Distance: 1.2080626294119547
Recognized Name: Unknown
1/1 [==============================] - 0s 38ms/step
Probability prediction: [[0.        0.        0.5987455 0.        0.4012545]]
Min Distance: 1.2033553955630485
Recognized Name: Unknown
1/1 [==============================] - 0s 52ms/step
Probability prediction: [[0.         0.         0.79896005 0.         0.20103995]]
Min Distance: 1.2234618241777455
Recognized Name: Unknown
1/1 [==============================] - 0s 61ms/step
Probability prediction: [[0. 0. 1. 0. 0.]]
Min Distance: 1.1485915307791699
Recognized Name: Unknown
1/1 [==============================] - 0s 43ms/step
Probability prediction: [[0. 0. 1. 0. 0.]]
Min Distance: 1.1363651735121993
Recognized Name: Unknown
1/1 [==============================] - 0s 38ms/step
Probability prediction: [[0. 0. 1. 0. 0.]]
Min Distance: 1.14668331987